# Vamos aprender a trabalhar com PDF usando o Python

- Regra geral: PDF foi feito justamente para bloquear muita coisa, então não é fácil "brincar" com um pdf
- Mesmo assim, Python tem várias bibliotecas que vão nos ajudar, vamos focar em 2:
    - PyPDF2
    - Tabula
- Ler e extrair informações de um PDF a gente consegue fazer.
- Escrever e Editar, aí já é outra história

### Para os nossos exemplos, vamos avaliar o Release de Resultados do 3º e 4º Trimestre de 2020 da Magazine Luiza

#### 1º Objetivo: Queremos conseguir separar apenas o DRE do Release de Resultados (Página 14) para enviar para a Diretoria, como fazemos?
    - Separar as páginas de um pdf

In [33]:
import PyPDF2 as pyf
from pathlib import Path
nome = 'MGLU_ER_3T20_POR.pdf'
arquivo_pdf = pyf.PdfReader(nome)

# print(arquivo_pdf)  # <PyPDF2._reader.PdfReader object at 0x000002D0B688CC10>

# print(len(arquivo_pdf.pages))  # 24

for i, pagina in enumerate(arquivo_pdf.pages):
    num_pagina = i + 1
    novo_pdf = pyf.PdfWriter()
    novo_pdf.add_page(pagina)
    with Path(f'paginas/Arquivo Pagina {num_pagina}.pdf').open(mode='wb') as arquivo: # atenção para a extensão do arquivo
        novo_pdf.write(arquivo)

#### 2º Objetivo: Com o Release de Resultados já separado página por página, queremos incluir apenas as Páginas de Destaque (Página 1), DRE (Página 14) e Balanço (Página 16).
    - Juntar vários pdfs em 1

In [34]:
import PyPDF2 as pyf
from pathlib import Path

num_paginas = [1, 14, 16]
novo_arquivo = pyf.PdfWriter()

for num in num_paginas:
    pagina_pdf = pyf.PdfReader(f'paginas/Arquivo Pagina {num}.pdf')
    novo_arquivo.add_page(pagina_pdf.pages[0])

with Path('paginas/Consolidado.pdf').open(mode='wb') as arquivo:  # atenção para a extensão do arquivo
    novo_arquivo.write(arquivo)

### Extra: Para adicionar todas as páginas de 2 pdfs

In [35]:
pdf_mesclado = pyf.PdfMerger()
pdf_mesclado.append('MGLU_ER_3T20_POR.pdf')
pdf_mesclado.append('MGLU_ER_4T20_POR.pdf')

with Path('paginas/Mesclado.pdf').open(mode='wb') as arquivo:  # atenção para a extensão do arquivo
    pdf_mesclado.write(arquivo)

# Funcionalidades que podem ser úteis:

- Inserir arquivo no meio do outro
- Quero colocar dentro do Resultado do 4T20 os destaques do 3T20 para poder comparar os 2 dentro do mesmo relatório

In [36]:
pdf_mesclado = pyf.PdfMerger()
pdf_mesclado.append('MGLU_ER_4T20_POR.pdf')
pdf_mesclado.merge(1, 'paginas/Arquivo Pagina 1.pdf')

with Path('paginas/Relatorio 2 Trimestres.pdf').open(mode='wb') as arquivo:  # atenção para a extensão do arquivo
    pdf_mesclado.write(arquivo)

- Rodar Página

In [37]:
arquivo_original = pyf.PdfReader('MGLU_ER_3T20_POR.pdf')
novo_arquivo = pyf.PdfWriter()

for pagina in arquivo_original.pages:
    pagina.rotate(90)
    novo_arquivo.add_page(pagina)
    
with Path('paginas/Paginas Rotacionadas.pdf').open(mode='wb') as arquivo:
    novo_arquivo.write(arquivo)

# Trabalhando com Textos e Informações Dentro do PDF

#### 1º Objetivo: Quero identificar como foram as Despesas com Vendas da MGLU
    - Pegar texto da página e identificar onde está essa informação

In [38]:
arquivo = pyf.PdfReader('MGLU_ER_3T20_POR.pdf')

qtd_paginas = len(arquivo.pages)
print(qtd_paginas)

metadados_arquivos = arquivo.metadata
print(metadados_arquivos)

24
{'/Title': 'DESEMPENHO FINANCEIRO CONSOLIDADO', '/Author': 'an_rezende', '/Subject': 'Receita Bruta', '/Creator': 'Microsoft® Office Word 2007', '/CreationDate': "D:20201109183121-03'00'", '/ModDate': "D:20201109183121-03'00'", '/Producer': 'Microsoft® Office Word 2007'}


In [39]:
texto_referencia = '| Despesas com Vendas'

for i, pagina in enumerate(arquivo.pages):
    texto_pagina = pagina.extract_text()
    if texto_referencia in texto_pagina:
        print('Numero da página: ',i+1)
        texto_analisar = texto_pagina

Numero da página:  10


In [40]:
print(texto_analisar)

Divulgação de Resultados  
3T20 
10 
 
  
 
| Despesas Operacionais  
 
R$ milhões  3T20  
Ajustado   % RL  3T19  
Ajustado   % RL   Var(%)  9M20  
Ajustado   % RL  9M19  
Ajustado   % RL   Var(%)  
  Despesas com Vendas   (1.432,6)  -17,2%  (890,0)  -18,3%  61,0%  (3.487,2)  -18,2%  (2.309,1)  -17,1%  51,0%  
  Despesas Gerais e Administrativas   (240,7)  -2,9%  (207,1)  -4,3%  16,2%  (617,3)  -3,2%  (498,2)  -3,7%  23,9%  
 Subtotal      (1.673,3)  -20,1%      (1.097,1)  -22,6%  52,5%      (4.104,5)  -21,5%      (2.807,4)  -20,8%  46,2%  
  Perdas em Liquidação Duvidosa   (25,4)  -0,3%  (20,2)  -0,4%  25,4%  (84,5)  -0,4%  (45,8)  -0,3%  84,3%  
  Outras Receitas Operacionais, Líquidas   15,2  0,2%  15,3  0,3%  -0,6%  41,0  0,2%  44,0  0,3%  -6,8%  
  Total de Despesas Operacionais       (1.683,5)  -20,3%      (1.102,0)  -22,7%  52,8%      (4.148,0)  -21,7%      (2.809,2)  -20,8%  47,7%  
 
| Despesas com Vendas  
 
No 3T20, as despesas com vendas totalizaram R$1.432,6 milhões, equiv

In [41]:
posicao_inicial = texto_analisar.find(texto_referencia)
posicao_final = texto_analisar.find('|', posicao_inicial + 1)
texto_final = texto_analisar[posicao_inicial:posicao_final]
print(texto_final)

| Despesas com Vendas  
 
No 3T20, as despesas com vendas totalizaram R$1.432,6 milhões, equivalentes a 17,2% da receita líquida, 1,1 p.p. menor que no 
3T19 , principalmente devido ao forte crescimento das vendas . Vale ressaltar que a Companhia conseguiu diluir as despesas com 
vendas m esmo investi ndo em maior nível de serviço,  especialmente em  atendimento e logística.  
 
Nos 9M20, as despesas com vendas totalizaram R$3.487,2 milhões, equivalentes a 18,2% da receita líquida (+1,1 p.p. versus  os 
9M19).  
 



#### 2º Objetivo: Quero analisar o DRE (sem ajuste - Página 5)
    - Para ler tabelas em pdf, use o tabula (é ninja)
    
    - Cuidado 1: Instale o tabula-py (não instale o tabula). Se instalar o tabula errado, desinstale ele, instale o tabula-py, desinstale o tabula-py e instale novamente o tabula-py. Reinicie o kernel do Jupyter após isso
    
    - Cuidado 2: Tem que ter o java instalado no seu computador (depois de instalar, reinicie o computador)

In [42]:
import tabula as tb

tabela = tb.read_pdf('MGLU_ER_3T20_POR.pdf', pages='5')
# print(tabela)  # retorna uma lista
df_resultado = tabela[0]
display(df_resultado)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,NaN,R$ milhões (exceto quando indicado),NaN,NaN,3T2,NaN,NaN,3T1,NaN,NaN,...,NaN,NaN,9M2,NaN,NaN,9M1,NaN,NaN,Var(%,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Vendas Totais1 (incluindo marketplace),NaN,NaN,"12.355,5",NaN,NaN,"6.817,6",NaN,NaN,...,NaN,NaN,"28.584,",NaN,NaN,"18.282,6",NaN,NaN,"56,3",NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Receita Bruta,NaN,NaN,"10.349,5",NaN,NaN,"5.999,4",NaN,NaN,...,NaN,NaN,"23.652,",NaN,NaN,"16.508,8",NaN,NaN,"43,3",NaN
5,NaN,Receita Líquida,NaN,NaN,"8.308,3",NaN,NaN,"4.864,2",NaN,NaN,...,NaN,NaN,"19.111,",NaN,NaN,"13.501,3",NaN,NaN,"41,6",NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,Lucro Bruto,NaN,NaN,"2.178,7",NaN,NaN,"1.424,9",NaN,NaN,...,NaN,NaN,"5.034,",NaN,NaN,"3.728,6",NaN,NaN,"35,0",NaN
8,NaN,Margem Bruta,NaN,NaN,"26,2",NaN,NaN,"29,3",NaN,NaN,...,NaN,NaN,"26,3",NaN,NaN,"27,6",NaN,NaN,"-1,3 p",NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
# excluir linhas totalmente vazias
df_resultado = df_resultado.dropna(how='all', axis=0)
# excluir colunas completamente vazias
df_resultado = df_resultado.dropna(how='all', axis=1)
display(df_resultado)

,Unnamed: 1,Unnamed: 4,Unnamed: 7,Unnamed: 10,Unnamed: 13,Unnamed: 16,Unnamed: 19
0,R$ milhões (exceto quando indicado),3T2,3T1,Var(%,9M2,9M1,Var(%
2,Vendas Totais1 (incluindo marketplace),"12.355,5","6.817,6","81,2","28.584,","18.282,6","56,3"
4,Receita Bruta,"10.349,5","5.999,4","72,5","23.652,","16.508,8","43,3"
5,Receita Líquida,"8.308,3","4.864,2","70,8","19.111,","13.501,3","41,6"
7,Lucro Bruto,"2.178,7","1.424,9","52,9","5.034,","3.728,6","35,0"
8,Margem Bruta,"26,2","29,3","-3,1 p","26,3","27,6","-1,3 p"
10,EBITDA,"546,1","501,2","9,0","1.022,","1.276,5","-19,9"
11,Margem EBITDA,"6,6","10,3","-3,7 p","5,3","9,5","-4,2 p"
13,Lucro Líquido,"206,0","235,1","-12,4","172,","753,8","-77,2"
14,Margem Líquida,"2,5","4,8","-2,3 p","0,9","5,6","-4,7 p"


In [44]:
# tornando a primeira linha como cabeçalho
df_resultado.columns = df_resultado.iloc[0]
df_resultado = df_resultado.iloc[1:]  # excluindo a linha de índice 0 que foi promovida a cabeçalho
df_resultado = df_resultado.reset_index(drop=True)  # dando reset no index, drop para não permitir que os índices virem coluna
display(df_resultado)

,R$ milhões (exceto quando indicado),3T2,3T1,Var(%,9M2,9M1,Var(%
0,Vendas Totais1 (incluindo marketplace),"12.355,5","6.817,6","81,2","28.584,","18.282,6","56,3"
1,Receita Bruta,"10.349,5","5.999,4","72,5","23.652,","16.508,8","43,3"
2,Receita Líquida,"8.308,3","4.864,2","70,8","19.111,","13.501,3","41,6"
3,Lucro Bruto,"2.178,7","1.424,9","52,9","5.034,","3.728,6","35,0"
4,Margem Bruta,"26,2","29,3","-3,1 p","26,3","27,6","-1,3 p"
5,EBITDA,"546,1","501,2","9,0","1.022,","1.276,5","-19,9"
6,Margem EBITDA,"6,6","10,3","-3,7 p","5,3","9,5","-4,2 p"
7,Lucro Líquido,"206,0","235,1","-12,4","172,","753,8","-77,2"
8,Margem Líquida,"2,5","4,8","-2,3 p","0,9","5,6","-4,7 p"
9,Lucro Bruto - Ajustado,"2.178,7","1.488,9","46,3","5.034,","3.964,6","27,0"


#### 3º Objetivo: Quero analisar o Capital de Giro e os Investimentos (ambas as tabelas na página 12)
    - Páginas com mais de 1 tabela

In [50]:
tabelas = tb.read_pdf('MGLU_ER_3T20_POR.pdf', pages=12)
# print(len(tabelas))  # 2 taebasl
for tabela in tabelas:
    # excluir linhas totalmente vazias
    tabela = tabela.dropna(how='all', axis=0)
    # excluir colunas completamente vazias
    tabela = tabela.dropna(how='all', axis=1)
    display(tabela)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,(+) Contas a Receber (sem Cartões de Crédito),"(26,7)","706,3","680,8","781,3","794,0","733,0"
1,(+) Estoques,"2.120,2","5.005,9","4.198,2","4.075,5","3.801,8","2.885,7"
2,(+) Partes Relacionadas (sem Cartão Luiza),"(10,5)","71,3","80,4","77,1","100,6","81,8"
3,(+) Impostos a Recuperar,"186,3","932,0","748,9","877,4","864,1","745,7"
4,(+) Outros Ativos,"(56,6)","88,5","100,2","143,5","136,3","145,1"
5,(+) Ativos Circulantes Operacionais,"2.212,7","6.804,0","5.808,6","5.954,8","5.696,8","4.591,3"
7,(-) Fornecedores,"2.301,5","6.104,3","5.334,0","4.132,7","5.934,9","3.802,8"
8,(-) Repasses e outros depósitos,"627,3","627,3","639,3","235,9",-,-
9,"(-) Salários, Férias e Encargos Sociais","95,0","444,7","329,0","263,3","354,7","349,8"
10,(-) Impostos a Recolher,"90,7","299,6","206,4","176,9","352,0","208,8"


,R$ milhões,3T20,%,3T19,%.1,Var(%),9M20,%.2,9M20.1,%.3,Var(%).1
1,Lojas Novas,"21,2",14%,"94,6",51%,-78%,"69,0",21%,"121,9",31%,-43%
2,Reformas,"6,1",4%,"8,2",4%,-26%,"14,6",4%,"38,2",10%,-62%
3,Tecnologia,"69,1",45%,"32,6",18%,112%,"147,6",45%,"84,4",22%,75%
4,Logística,"36,3",24%,"32,8",18%,11%,"62,1",19%,"107,4",27%,-42%
5,Outros,"21,5",14%,"17,7",10%,22%,"35,7",11%,"38,8",10%,-8%
6,Total,"154,2",100%,"186,0",100%,-17%,"329,1",100%,"390,7",100%,-16%


In [51]:
df_capitalgiro = tabelas[0]
df_capitalgiro = df_capitalgiro.dropna(how='all', axis=0)
df_capitalgiro = df_capitalgiro.dropna(how='all', axis=1)
# display(df_capitalgiro)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,(+) Contas a Receber (sem Cartões de Crédito),"(26,7)","706,3","680,8","781,3","794,0","733,0"
1,(+) Estoques,"2.120,2","5.005,9","4.198,2","4.075,5","3.801,8","2.885,7"
2,(+) Partes Relacionadas (sem Cartão Luiza),"(10,5)","71,3","80,4","77,1","100,6","81,8"
3,(+) Impostos a Recuperar,"186,3","932,0","748,9","877,4","864,1","745,7"
4,(+) Outros Ativos,"(56,6)","88,5","100,2","143,5","136,3","145,1"
5,(+) Ativos Circulantes Operacionais,"2.212,7","6.804,0","5.808,6","5.954,8","5.696,8","4.591,3"
7,(-) Fornecedores,"2.301,5","6.104,3","5.334,0","4.132,7","5.934,9","3.802,8"
8,(-) Repasses e outros depósitos,"627,3","627,3","639,3","235,9",-,-
9,"(-) Salários, Férias e Encargos Sociais","95,0","444,7","329,0","263,3","354,7","349,8"
10,(-) Impostos a Recolher,"90,7","299,6","206,4","176,9","352,0","208,8"


#### O que fazer quando o tabula não consegue ler alguma linha da tabela? Como o cabeçalho, no nosso caso?

In [54]:
tabelas2 = tb.read_pdf('MGLU_ER_3T20_POR.pdf', pages=12, lattice=True)  # lattice=True ***
df_capitalgiro2 = tabelas2[0]
df_capitalgiro2 = df_capitalgiro2.dropna(how='all', axis=0)
df_capitalgiro2 = df_capitalgiro2.dropna(how='all', axis=1)
df_capitalgiro2 = df_capitalgiro2.reset_index(drop=True)
display(df_capitalgiro2)

,Unnamed: 2,Unnamed: 5,Unnamed: 6,Unnamed: 8,Unnamed: 9,Unnamed: 11,Unnamed: 12,Unnamed: 14,Unnamed: 15,Unnamed: 17,Unnamed: 18,Unnamed: 20,Unnamed: 21
0,R$ milhões,Dif 12U,NaN,set-2,NaN,jun-2,NaN,mar-2,NaN,dez-1,NaN,set-1,NaN
1,NaN,NaN,"(26,7",NaN,"706,3",NaN,"680,8",NaN,"781,3",NaN,"794,0",NaN,"733,0"
2,(+) Contas a Receber (sem Cartões de Crédito),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,"2.120,2",NaN,"5.005,9",NaN,"4.198,2",NaN,"4.075,5",NaN,"3.801,8",NaN,"2.885,7"
4,(+) Estoques,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,"(10,5",NaN,"71,3",NaN,"80,4",NaN,"77,1",NaN,"100,6",NaN,"81,8"
6,(+) Partes Relacionadas (sem Cartão Luiza),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,"186,3",NaN,"932,0",NaN,"748,9",NaN,"877,4",NaN,"864,1",NaN,"745,7"
8,(+) Impostos a Recuperar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,"(56,6",NaN,"88,5",NaN,"100,2",NaN,"143,5",NaN,"136,3",NaN,"145,1"


In [56]:
colunas = df_capitalgiro2.iloc[0]
colunas = colunas.dropna()
display(colunas)

Unnamed: 2     R$ milhões
Unnamed: 5        Dif 12U
Unnamed: 8          set-2
Unnamed: 11         jun-2
Unnamed: 14         mar-2
Unnamed: 17         dez-1
Unnamed: 20         set-1
Name: 0, dtype: object

In [57]:
df_capitalgiro.columns = colunas
display(df_capitalgiro)

,R$ milhões,Dif 12U,set-2,jun-2,mar-2,dez-1,set-1
0,(+) Contas a Receber (sem Cartões de Crédito),"(26,7)","706,3","680,8","781,3","794,0","733,0"
1,(+) Estoques,"2.120,2","5.005,9","4.198,2","4.075,5","3.801,8","2.885,7"
2,(+) Partes Relacionadas (sem Cartão Luiza),"(10,5)","71,3","80,4","77,1","100,6","81,8"
3,(+) Impostos a Recuperar,"186,3","932,0","748,9","877,4","864,1","745,7"
4,(+) Outros Ativos,"(56,6)","88,5","100,2","143,5","136,3","145,1"
5,(+) Ativos Circulantes Operacionais,"2.212,7","6.804,0","5.808,6","5.954,8","5.696,8","4.591,3"
7,(-) Fornecedores,"2.301,5","6.104,3","5.334,0","4.132,7","5.934,9","3.802,8"
8,(-) Repasses e outros depósitos,"627,3","627,3","639,3","235,9",-,-
9,"(-) Salários, Férias e Encargos Sociais","95,0","444,7","329,0","263,3","354,7","349,8"
10,(-) Impostos a Recolher,"90,7","299,6","206,4","176,9","352,0","208,8"


# Outro método que pode ser útil algum dia: Captar Imagem em um pdf
    - biblioteca pikepdf

In [59]:
from pikepdf import Pdf, PdfImage

arquivo = Pdf.open('MGLU_ER_3T20_POR.pdf')
for pagina in arquivo.pages:
    #imagens = pagina.images.items()  # <pikepdf._core.ItemsView[str, QPDFObjectHandle] object at 0x0000022BD812E7F0>
    for nome, imagem in pagina.images.items():
        imagem_salvar = PdfImage(imagem)
        imagem_salvar.extract_to(fileprefix=f'imagens/{nome}')

# Substituir texto no pdf tipo contrato

- Não recomendo fazer diretamente pelo Python. Realmente do que vi a melhor opção me parece o Word fazer isso
- Caso precise automatizar, automatize o processo fazendo ele pelo Word
- Quem quiser MUITO fazer isso pelo Python, tem um link aqui que vai te ajudar de uma solução que achei que funciona. Tem seus bugs/cuidados especiais, mas funciona: https://pdf.co/samples/pdf-co-web-api-replace-text-from-pdf-python-replace-text-from-url